# Import

In [1]:
import json
from jovis_model.config import Config
from run import ModelRunner

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# CLIP TEST

In [1]:
from jovis_model.models.llm.mclip import CLIPModel

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
params = {
    "pkg": "llm",
    "task": "mclip",
    "use_hf_model": True,
    "params": {
        "hf_name": "M-CLIP/XLM-Roberta-Large-Vit-B-32"
    }
}
config = Config(**params)
model = CLIPModel(config)

In [ ]:
model, preprocess = clip.load("ViT-B/32")

100%|███████████████████████████████████████| 338M/338M [00:34<00:00, 10.3MiB/s]


In [ ]:
outputs = model.inference(["안녕하세요"])

In [ ]:
file_path = "./jovis_model/_db/llm/multimodel_test/export_data_skb.json"
with open(file_path, "r") as f:
    data = json.load(f)

In [ ]:
from PIL import Image

In [ ]:
data["693669"]["s3_url"]

'https://flush-image-prod.s3.ap-northeast-2.amazonaws.com/01H5S0JRNJEDPD3KX414FBPH2K/693669.webp'

In [ ]:
image = Image.open(data["693669"]["s3_url"]).convert("RGB")

FileNotFoundError: [Errno 2] No such file or directory: '/home/omnious/workspace/jovis/jovis-model/https:/flush-image-prod.s3.ap-northeast-2.amazonaws.com/01H5S0JRNJEDPD3KX414FBPH2K/693669.webp'

# NER_TEST

In [2]:
# params = {
#     "pkg": "llm",
#     "task": "chat",
#     "use_hf_model": True,
#     "params": {
#         "hf_name": "meta-llama/Meta-Llama-3-8B-Instruct"
#     }
# }

params = {
    "pkg": "llm",
    "task": "bedrock",
    "use_hf_model": False,
    "params": {
    }
}
config = Config(**params)
runner = ModelRunner(
    config=config,
    mode="inference"
)

In [5]:
dialogs = [
    {"role": "user", "content": "What is you name?"}
]

In [6]:
out = runner.run(
    sample_inputs=dialogs
)

# Test

In [2]:
params = {
    "pkg": "llm",
    "task": "chat",
    "use_hf_model": True,
    "params": {
        "hf_name": "OpenGVLab/InternVL-Chat-V1-5"
    }
}

In [3]:
config = Config(**params)

In [4]:
from transformers import AutoModel
from jovis_model.models.llm.chat import ChatModel

In [5]:
processor = ChatModel(config=config)

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 11/11 [01:34<00:00,  8.61s/it]
/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
processor.model

InternVLChatModel(
  (vision_model): InternVisionModel(
    (embeddings): InternVisionEmbeddings(
      (patch_embedding): Conv2d(3, 3200, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): InternVisionEncoder(
      (layers): ModuleList(
        (0): InternVisionEncoderLayer(
          (attn): InternAttention(
            (qkv): Linear(in_features=3200, out_features=9600, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (q_norm): InternRMSNorm()
            (k_norm): InternRMSNorm()
            (inner_attn): FlashAttention()
            (proj): Linear(in_features=3200, out_features=3200, bias=True)
          )
          (mlp): InternMLP(
            (act): GELUActivation()
            (fc1): Linear(in_features=3200, out_features=12800, bias=True)
            (fc2): Linear(in_features=12800, out_features=3200, bias=True)
          )
          (norm1): InternRMSNorm()
          (norm2): InternR